In [87]:
import torch
print(torch.__version__)
from math import log

2.1.2+cpu


In [104]:
D = 4
H = 8
R0 = 0.1
op = int(1e5)
batch = 10
hidden_size = 100
opt = 0.01

In [105]:
from collections import OrderedDict

model = torch.nn.Sequential(
    OrderedDict([
        ("linear_1", torch.nn.Linear(D * H, hidden_size)),
        ("activation_1", torch.nn.ReLU()),
        ("linear_2", torch.nn.Linear(hidden_size, D + 1))
    ])
)
log_Z = torch.nn.Parameter(torch.tensor(0.0, requires_grad=True))
log_Z_optimizer = torch.optim.Adam([log_Z], opt)

In [106]:
optimizer = torch.optim.Adam(
    model.parameters(),  # Передаём все тензоры, учавствующие в градиентном спуске
    lr=0.0001,  # learning rate - величина шага градиентного спуска
)


In [107]:
def RIGHT_staaate(state):
    global H, D, batch
    res = [[float(0)] * H * D for i in range(batch)]
    for k in range(0, batch):
        for i in range(0, D):
            res[k][state[k][i] + i * H] = float(1)
    return res

In [108]:
def reward(x):
    global R0, D, H
    f1 = True
    f2 = True
    for d in range(D):
        if not (0.25 < abs(x[d] / (H - 1) - 0.5) <= 0.5): f1 = False
        if not (0.3 < abs(x[d] / (H - 1) - 0.5) < 0.4): f2 = False
    return (R0 + 0.5 * f1 + 2 * f2)


In [109]:
def loss_fn():
    global H, D, log_Z, batch
    used = [0] * batch
    allstate = [[0] * D for i in range(batch)]
    way = [[]] * batch
    while True:
        p_Fs = model(torch.tensor(RIGHT_staaate(allstate)))
        was = False
        for j in range(0, batch):
            if used[j]:
                continue
            was = True
            for i in range(0, D):
                if allstate[j][i] == H - 1:
                    p_Fs[j][i] = -float("inf")
        if was == False:
            break
        m = torch.nn.Softmax(dim=1)
        p_Fs = m(p_Fs)
        while True:
            ind = torch.distributions.categorical.Categorical(p_Fs).sample()
            iscorrect = True
            for j in range(0, batch):
                if used[j] or ind[j] == D:
                    continue
                if allstate[j][ind[j]] == H - 1:
                    iscorrect = False
            if iscorrect == False:
                continue
            for j in range(0, batch):
                if used[j]:
                    continue
                p_B = 0
                for i in range(0, D):
                    if allstate[j][i] >= 1:
                        p_B += 1
                p_B = max(p_B, 1)
                way[j].append([torch.log(p_Fs[j][ind[j]]), log(1 / p_B)])
                if ind[j] == D:
                    #way[j][-1][1] = 1
                    way.append([0, 0])
                    used[j] = True
                else:
                    assert(allstate[j][ind[j]] != H - 1)
                    allstate[j][ind[j]] += 1
            break
    j = 0
    loss = 0
    for state in allstate:
        cur = log_Z - log(reward(state))
        for i in range(0, len(way[j]) - 1):
            cur += way[j][i][0] - way[j][i + 1][1]
        cur = cur ** 2
        loss += cur
        j += 1
    loss /= batch
    return loss, allstate

In [110]:
from itertools import product

rewards = torch.zeros(*[H for i in range(D)])
coord_diap = [range(H) for _ in range(D)]

for coord in product(*coord_diap):
    rewards[tuple(coord)] = reward(torch.tensor(coord))

rewards /= rewards.sum()

def loss2(a, samples=1000):
    counter = torch.zeros(*[H for i in range(D)])
    for i in range(samples):
        counter[tuple(a[i])] += 1

    counter /= counter.sum()

    return (rewards - counter).abs().sum()


In [ ]:
a = []
for i in range(op):
    optimizer.zero_grad()
    log_Z_optimizer.zero_grad()
    loss = 0
    x = loss_fn()
    loss = x[0]
    for j in x[1]:
        a.append(j)
    loss.backward()
    log_Z_optimizer.step()
    optimizer.step()
    if i % 100 == 0:
        print(i * batch)
        assert(torch.tensor(a[-10000:]).max() <= 7)
        print(loss2(a[-10000:], len(a[-10000:])))

0
tensor(1.9961)
1000
tensor(1.8600)
2000
tensor(1.7973)
3000
tensor(1.7394)
4000
tensor(1.6974)
5000
tensor(1.6599)
6000
tensor(1.6239)
7000
tensor(1.6105)
8000
tensor(1.5994)
9000
tensor(1.5837)
10000
tensor(1.5699)
11000
tensor(1.5553)
12000
tensor(1.5388)
13000
tensor(1.5227)
14000
tensor(1.5062)
15000
tensor(1.4930)
16000
tensor(1.4768)
17000
tensor(1.4633)
18000
tensor(1.4527)
19000
tensor(1.4390)
20000
tensor(1.4231)
21000
tensor(1.4163)
22000
tensor(1.4105)
23000
tensor(1.4064)
24000
tensor(1.4053)
25000
tensor(1.4016)
26000
tensor(1.3999)
